# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 56 new papers today
          21 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/20 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2210.01218


extracting tarball to tmp_2210.01218...

 done.
Retrieving document from  https://arxiv.org/e-print/2210.01228


extracting tarball to tmp_2210.01228...

...

extracting tarball to tmp_2210.01666...

 done.
Retrieving document from  https://arxiv.org/e-print/2210.01755


/opt/hostedtoolcache/Python/3.9.14/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3378: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.14/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3378: LatexWarning: Found documentclass in tmp_2210.01666/main.tex

  exec(code_obj, self.user_global_ns, self.user_ns)


extracting tarball to tmp_2210.01755...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.01556-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.01556) | **Nuclear star cluster formation in star-forming dwarf galaxies**  |
|| Katja Fahrion, et al. -- incl., <mark>Oliver Müller</mark>, <mark>Nadine Neumayer</mark>, <mark>Francesca Pinna</mark> |
|*Appeared on*| *2022-10-05*|
|*Comments*| *13 pages, 6 figures, accepted for publication in Astronomy & Astrophysics*|
|**Abstract**| Nuclear star clusters (NSCs) are massive star clusters found in all types of galaxies from dwarfs to massive galaxies. Recent studies show that while low-mass NSCs in dwarf galaxies ($M_\text{gal} < 10^{9} M_\odot$) form predominantly out of the merger of globular clusters (GCs), high-mass NSCs in massive galaxies have assembled most of their mass through central enriched star formation. So far, these results of a transition in the dominant NSC formation channel have been based on studies of early-type galaxies and massive late-type galaxies. Here, we present the first spectroscopic analysis of a sample of nine nucleated late-type dwarf galaxies with the aim of identifying the dominant NSC formation pathway. We use integral-field spectroscopy data obtained with the Multi Unit Spectroscopic Explorer (MUSE) instrument to analyse the ages, metallicities, star formation histories, and star formation rates of the NSCs and their surroundings. Our sample includes galaxies with stellar masses $M_\text{gal} = 10^7 - 10^9 M_\odot$ and NSC masses $M_\text{NSC} = 6 \times 10^4 - 6 \times 10^{6} M_\odot$. Although all NSC spectra show emission lines, this emission is not always connected to star formation within the NSC, but rather to other regions along the line-of-sight. The NSC star formation histories reveal that metal-poor and old populations dominate the stellar populations in five NSCs, possibly stemming from the inspiral of GCs. The NSCs of the most massive galaxies in our sample show significant contributions from young and enriched populations that indicate additional mass growth through central star formation. Our results support previous findings of a transition in the dominant NSC formation channel with galaxy mass, showing that the NSCs in low-mass galaxies predominantly grow through the inspiral of GCs, while central star formation can contribute to NSC growth in more massive galaxies. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.01755-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.01755) | **Late-time H/He-poor circumstellar interaction in the normal-looking  type-Ic supernova SN 2021ocs: an exposed oxygen-magnesium layer and extreme  stripping of the progenitor**  |
|| H. Kuncarayakti, et al. -- incl., <mark>M. E. Huber</mark>, <mark>K. W. Smith</mark> |
|*Appeared on*| *2022-10-05*|
|*Comments*| *9 pages, 5 figures, submitted*|
|**Abstract**| Supernova (SN) 2021ocs was discovered in the galaxy NGC 7828 within the interacting system Arp 144, and subsequently classified as a normal type-Ic SN around peak brightness. VLT/FORS2 observations in the nebular phase at 150 d reveal that the spectrum is dominated by oxygen and magnesium emission lines of different transitions and ionization states: O I, [O I], [O II], [O III], Mg I, and Mg II. Such a spectrum has no counterpart in the literature, though it bears a few features similar to those of some interacting type Ibn and Icn SNe. Additionally, SN 2021ocs showed a blue color, $(g-r) \lesssim -0.5$ mag, after the peak, atypical for a type-Ic SN. Together with the nebular spectrum, this suggests that SN 2021ocs underwent late-time interaction with an H/He-poor circumstellar medium (CSM), resulting from the pre-SN progenitor mass loss during its final $\sim$1000 days. The strong O and Mg lines and the absence of strong C and He lines suggest that the progenitor star's O-Mg layer is exposed, which places SN 2021ocs as the most extreme case of massive progenitor star's envelope stripping in interacting SNe, followed by type-Icn (stripped C-O layer) and Ibn (stripped He-rich layer) SNe. This is the first time such case is reported in the literature. SN 2021ocs emphasizes the importance of late-time spectroscopy of even seemingly normal SNe, which reveals the inner ejecta and progenitor star's mass loss history. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.01218-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.01218) | **Unraveling Twisty Linear Polarization Morphologies in Black Hole Images**  |
|| Razieh Emami, et al. -- incl., <mark>Lindy Blackburn</mark>, <mark>Matthew Liska</mark>, <mark>Lars Hernquist</mark>, <mark>Randall Smith</mark> |
|*Appeared on*| *2022-10-05*|
|*Comments*| *25 pages, 19 figures*|
|**Abstract**| We investigate general relativistic magnetohydrodynamic simulations (GRMHD) to determine the physical origin of the twisty patterns of linear polarization seen in spatially resolved black hole images and explain their morphological dependence on black hole spin. By characterising the observed emission with a simple analytic ring model, we find that the twisty morphology is determined by the magnetic field structure in the emitting region. Moreover, the dependence of this twisty pattern on spin can be attributed to changes in the magnetic field geometry that occur due to the frame dragging. By studying an analytic ring model, we find that the roles of Doppler boosting and lensing are subdominant. Faraday rotation may cause a systematic shift in the linear polarization pattern, but we find that its impact is subdominant for models with strong magnetic fields and modest ion-to-electron temperature ratios. Models with weaker magnetic fields are much more strongly affected by Faraday rotation and have more complicated emission geometries than can be captured by a ring model. However, these models are currently disfavoured by the recent EHT observations of M87*. Our results suggest that linear polarization maps can provide a probe of the underlying magnetic field structure around a black hole, which may then be usable to indirectly infer black hole spins. The generality of these results should be tested with alternative codes, initial conditions, and plasma physics prescriptions. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.01544-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.01544) | **The First Survey of Quiet Sun Features Observed in Hard X-Rays With  NuSTAR**  |
|| <mark>Sarah Paterson</mark>, et al. -- incl., <mark>Lindsay Glesener</mark>, <mark>David M. Smith</mark> |
|*Appeared on*| *2022-10-05*|
|*Comments*| **|
|**Abstract**| We present the first survey of quiet Sun features observed in hard X-rays (HXRs), using the the Nuclear Spectroscopic Telescope ARray (NuSTAR), a HXR focusing optics telescope. The recent solar minimum combined with NuSTAR's high sensitivity has presented a unique opportunity to perform the first HXR imaging spectroscopy on a range of features in the quiet Sun. By studying the HXR emission of these features we can detect or constrain the presence of high temperature (>5 MK) or non-thermal sources, to help understand how they relate to larger more energetic solar phenomena, and determine their contribution to heating the solar atmosphere. We report on several features observed in the 28 September 2018 NuSTAR full-disk quiet Sun mosaics, the first of the NuSTAR quiet Sun observing campaigns, which mostly include steady features of X-ray bright points and an emerging flux region which later evolved into an active region, as well as a short-lived jet. We find that the features' HXR spectra are well fitted with isothermal models with temperatures ranging between 2.0-3.2 MK. Combining the NuSTAR data with softer X-ray emission from Hinode/XRT and EUV from SDO/AIA we recover the differential emission measures, confirming little significant emission above 4 MK. The NuSTAR HXR spectra allow us to constrain the possible non-thermal emission that would still be consistent with a null HXR detection. We found that for only one of the features (the jet) was there a potential non-thermal upper limit capable of powering the heating observed. However, even here the non-thermal electron distribution had to be very steep (effectively mono-energetic) with a low energy cut-off between 3-4 keV. The higher temperature or non-thermal sources in the typical quiet Sun features found in this September 2018 data are therefore found to be very weak, if present at all. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.01576-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.01576) | **Star formation timescale in the molecular filament WB 673**  |
|| O. L. Ryabukhina, M. S. Kirsanova, <mark>C. Henkel</mark>, D. S. Wiebe |
|*Appeared on*| *2022-10-05*|
|*Comments*| *10 pages, 7 figures, accepted for publication in MNRAS*|
|**Abstract**| We present observations of ammonia emission lines toward the interstellar filament WB~673 hosting the dense clumps WB~673, WB~668, S233-IR and G173.57+2.43. LTE analysis of the lines allows us to estimate gas kinetic temperature ($\lesssim$ 30~K in all the clumps), number density ($7-17\times10^3$~cm$^{-3}$), and ammonia column density ($\approx 1-1.5\times 10^{15}$~cm$^{-2}$) in the dense clumps. We find signatures of collapse in WB 673 and presence of compact spatially unresolved dense clumps in S233-IR. We reconstruct 1D density and temperature distributions in the clumps and estimate their ages using astrochemical modelling. Considering CO, CS, NH$_3$ and N$_2$H$^+$ molecules (plus HCN and HNC for WB~673), we find a chemical age of $t_{\rm chem}=1-3\times 10^5$~yrs providing the best agreement between the simulated and observed column densities in all the clumps. Therefore, we consider $t_{\rm chem}$ as the chemical age of the entire filament. A long preceding low-density stage of gas accumulation in the astrochemical model would break the agreement between the simulated and observed column densities. We suggest that rapid star formation over a $\sim 10^5$~yrs timescale take place in the filament. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.01666-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.01666) | **Neural Network Based Point Spread Function Deconvolution For  Astronomical Applications**  |
|| Hong Wang, et al. -- incl., <mark>Yuewei Lin</mark> |
|*Appeared on*| *2022-10-05*|
|*Comments*| *12 pages, 6 figures*|
|**Abstract**| Optical astronomical images are strongly affected by the point spread function (PSF) of the optical system and the atmosphere (seeing) which blurs the observed image. The amount of blurring depends both on the observed band, and more crucially, on the atmospheric conditions during observation. A typical astronomical image will therefore have a unique PSF that is non-circular and different in different bands. Observations of known stars give us a determination of this PSF. Therefore, any serious candidate for production analysis of astronomical images must take the known PSF into account during the image analysis. So far the majority of applications of neural networks (NN) to astronomical image analysis have ignored this problem by assuming a fixed PSF in training and validation. We present a neural network based deconvolution algorithm based on Deep Wiener Deconvolution Network (DWDN) that takes the PSF shape into account when performing deconvolution as an example of one possible approach to enabling neural network to use the PSF information. We study the performance of several versions of this algorithm under realistic observational conditions in terms of recovery of most relevant astronomical quantities such as colors, ellipticities and orientations. We also investigate the performance of custom loss functions and find that they cause modest improvements in the recovery of astronomical quantities. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.01608-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.01608) | **$\textit{Gaia}$ white dwarfs within 40 pc III: spectroscopic  observations of new candidates in the southern hemisphere**  |
|| Mairi W. O'Brien, et al. -- incl., <mark>N. P. Gentile Fusillo</mark>, <mark>J. J. Hermes</mark>, <mark>B. L. Klein</mark>, <mark>S. J. Kleinman</mark>, <mark>M. R. Schreiber</mark>, <mark>S. Torres</mark> |
|*Appeared on*| *2022-10-05*|
|*Comments*| *49 pages, 19 figures. Submitted to MNRAS on 4 October, 2022*|
|**Abstract**| We present a spectroscopic survey of 248 white dwarf candidates within 40 pc of the Sun; of these 244 are in the southern hemisphere. Observations were performed mostly with the Very Large Telescope (X-Shooter) and Southern Astrophysical Research Telescope. Almost all candidates were selected from $\textit{Gaia}$ Data Release 3 (DR3). We find a total of 246 confirmed white dwarfs, 209 of which had no previously published spectra, and two main-sequence star contaminants. Of these, 100 white dwarfs display hydrogen Balmer lines, 69 have featureless spectra, and two show only neutral helium lines. Additionally, 14 white dwarfs display traces of carbon, while 37 have traces of other elements that are heavier than helium. We observe 36 magnetic white dwarfs through the detection of Zeeman splitting of their hydrogen Balmer or metal spectral lines. High spectroscopic completeness (> 97 per cent) has now been reached, such that we have 1058 confirmed $\textit{Gaia}$ DR3 white dwarfs out of 1083 candidates within 40 pc of the Sun at all declinations. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Max' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.01291-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.01291) | **Seismic Monitoring of the Sun's Far Hemisphere: A Crucial Component in  Future Space Weather Forecasting (A White Paper Submitted to the Decadal  Survey for Solar and Space Physics (Heliophysics) -- SSPH 2024-2033)**  |
|| Kiran Jain, et al. -- incl., <mark>C. Lindsey</mark>, <mark>C. J. Henney</mark> |
|*Appeared on*| *2022-10-05*|
|*Comments*| **|
|**Abstract**| The purpose of this white paper is to put together a coherent vision for the role of helioseismic monitoring of magnetic activity in the Sun's far hemisphere that will contribute to improving space weather forecasting as well as fundamental research in the coming decade. Our goal fits into the broader context of helioseismology in solar research for any number of endeavors when helioseismic monitors may be the sole synoptic view of the Sun's far hemisphere. It is intended to foster a growing understanding of solar activity, as realistically monitored in both hemispheres, and its relationship to all known aspects of the near-Earth and terrestrial environment. Some of the questions and goals that can be fruitfully pursued through seismic monitoring of farside solar activity in the coming decade include: What is the relationship between helioseismic signatures and their associated magnetic configurations, and how is this relationship connected to the solar EUV irradiance over the period of a solar rotation?; How can helioseismic monitoring contribute to data-driven global magnetic-field models for precise space weather forecasting?; What can helioseismic monitors tell us about prospects of a flare, CME or high-speed stream that impacts the terrestrial environment over the period of a solar rotation?; How does the inclusion of farside information contribute to forecasts of interplanetary space weather and the environments to be encountered by human crews in interplanetary space? Thus, it is crucial for the development of farside monitoring of the Sun be continued into the next decade either through ground-based or space-borne observations. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2210.01556.md
    + _build/html/tmp_2210.01556/./Figures/galaxy_starformationrates_small.png
    + _build/html/tmp_2210.01556/./Figures/galaxy_sample.png
    + _build/html/tmp_2210.01556/./Figures/NSC_spectra.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand$</div>



<div id="title">

# Based on observations collected at the ESO Paranal La Silla Observatory, Chile, Prog. ID 108.220K.001, 0104.D-0503, 0100.B-0116, and 095.B-0.532.

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2210.01556-b31b1b.svg)](https://arxiv.org/abs/2210.01556)<mark>Appeared on: 2022-10-05</mark> - _13 pages, 6 figures, accepted for publication in Astronomy & Astrophysics_

</div>
<div id="authors">

Katja Fahrion, et al. -- incl., <mark><mark>Oliver Müller</mark></mark>, <mark><mark>Nadine Neumayer</mark></mark>, <mark><mark>Francesca Pinna</mark></mark>

</div>
<div id="abstract">

**Abstract:** Nuclear star clusters (NSCs) are massive star clusters found in all types of galaxies from dwarfs to massive galaxies. Recent studies show that while low-mass NSCs in dwarf galaxies ($M_\text{gal} < 10^{9} M_\odot$) form predominantly out of the merger of globular clusters (GCs), high-mass NSCs in massive galaxies have assembled most of their mass through central enriched star formation. So far, these results of a transition in the dominant NSC formation channel have been based on studies of early-type galaxies and massive late-type galaxies. Here, we present the first spectroscopic analysis of a sample of nine nucleated late-type dwarf galaxies with the aim of identifying the dominant NSC formation pathway. We use integral-field spectroscopy data obtained with the Multi Unit Spectroscopic Explorer (MUSE) instrument to analyse the ages, metallicities, star formation histories, and star formation rates of the NSCs and their surroundings. Our sample includes galaxies with stellar masses$M_\text{gal} = 10^7 - 10^9 M_\odot$and NSC masses$M_\text{NSC} = 6 \times 10^4 - 6 \times 10^{6} M_\odot$. Although all NSC spectra show emission lines, this emission is not always connected to star formation within the NSC, but rather to other regions along the line of sight. The NSC star formation histories reveal that metal-poor and old populations dominate the stellar populations in five NSCs, possibly stemming from the inspiral of GCs. The NSCs of the most massive galaxies in our sample show significant contributions from young and enriched populations that indicate additional mass growth through central star formation. Our results support previous findings of a transition in the dominant NSC formation channel with galaxy mass, showing that the NSCs in low-mass galaxies predominantly grow through the inspiral of GCs, while central star formation can contribute to NSC growth in more massive galaxies.

</div>

<div id="div_fig1">

<img src="tmp_2210.01556/./Figures/galaxy_starformationrates_small.png" alt="Fig3" width="100%"/>

**Figure 3. -** Star formation rate surface densities $\Sigma_{\text{SFR}}$ for the different galaxies. Each panel shows the 10$\arcsec$$\times$ 10$\arcsec$  zoom to the region around the NSCs. The positions of the NSCs are highlighted by the circles. Each circle has a radius corresponding to the PSF FWHM. The dashed circles highlight the circumnuclear region at 2$\arcsec$, defined by an annulus with inner radius of 8 pixel = 1.6$\arcsec$ and outer radius of 13 pixel = 2.6$\arcsec$. Regions with low signal-to-noise in the H$\alpha$ flux were masked. (*fig:SFRs*)

</div>
<div id="div_fig2">

<img src="tmp_2210.01556/./Figures/galaxy_sample.png" alt="Fig4" width="100%"/>

**Figure 4. -** RGB images of the sample galaxies based on synthetic $V$, $I$, and $R$ band images obtained from the MUSE cubes. The inset shows a zoom to a 10$\arcsec$$\times$ 10$\arcsec$ region around each NSC. The conversion from arcseconds to kpc is shown in white. North is up, east is to the left. (*fig:galaxy_sample*)

</div>
<div id="div_fig3">

<img src="tmp_2210.01556/./Figures/NSC_spectra.png" alt="Fig1" width="100%"/>

**Figure 1. -** NSC spectra in black with $\textsc${pPXF} fit in orange, showing a fit of both the absorption and emission lines. The light purple regions were masked during the fits. Each panel corresponds to one galaxy. We note that the y-axis scale is not the same in all panels. The range is chosen to highlight the absorption line features. For observations acquired with AO, the region around the Na D line is blocked. (*fig:NSC_spectra*)

</div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

55  publications in the last 7 days.
	 _build/html/2210.01556.md
	 _build/html/2210.00863.md
	 _build/html/2210.00560.md
	 _build/html/2209.15403.md
	 _build/html/2209.15110.md
	 _build/html/2209.14410.md
	 _build/html/2209.14396.md
	 _build/html/2209.13717.md
	 _build/html/2209.13336.md
	 _build/html/2209.12906.md
	 _build/html/2209.11781.md
	 _build/html/2209.11602.md
	 _build/html/2209.11248.md
	 _build/html/2209.11096.md
	 _build/html/2209.10618.md
	 _build/html/2209.09914.md
	 _build/html/2209.09823.md
	 _build/html/2209.08242.md
	 _build/html/2209.08120.md
	 _build/html/2209.08106.md
	 _build/html/2209.07962.md
	 _build/html/2209.07931.md
	 _build/html/2209.06833.md
	 _build/html/2209.05814.md
	 _build/html/2209.05593.md
	 _build/html/2209.04304.md
	 _build/html/2209.03367.md
	 _build/html/2209.02872.md
	 _build/html/2209.02725.md
	 _build/html/2209.02722.md
	 _build/html/2209.02092.md
	 _build/html/2209.01125.md
	 _build/html/2209.00828.md
	 _build/html/2209.00363.md
	 _build/ht

In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

# Debugging papers